In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

class NaiveBayes:

    def __init__(self):
      self._priors = None
      self._classes = None
      self._class_prob = None

    def fit(self, X, y):
      self.priors(X, y)
      self.class_prob(X, y)
      return

    def classify(self, x):
      pc1 = 1
      pc2 = 1
      count=0
      for i in x.index:
        if(x[count] not in self._class_prob[i]['-1']):
            pc1 *= 0.001  
        else:
            pc1 *= self._class_prob[i]['-1'][x[count]]
        if(x[count] not in self._class_prob[i]['1']):
            pc2 *= 0.001  
        else:
            pc2 *= self._class_prob[i]['1'][x[count]]
        count+=1
      if pc1/(0.23**10) >= pc2/(0.76**10):
        return -1 
      elif pc2/(0.76**10) > pc2/(0.23**10):
        return 1
      else:
        return 1
      return 1

    def class_prob(self, X, y):
      print("reached here")
      self._class_prob = {}
      df = pd.concat([X,y], axis=1)
      i=0
      for col in df.columns:
        if i%1000==0 and i!=0:
            print(i)
        self._class_prob[col] = df.groupby(['salary', col]).size().add(0)
        self._class_prob[col]/=len(df) 
        i+=1
      return

    def priors(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        
        # calculate prior for each class
        self._priors = np.zeros(n_classes, dtype=np.float64)
        for idx, c in enumerate(self._classes):
            X_c = X.loc[y == str(c)]
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = []
        for i in range(X.shape[0]):
          if i%1000 == 0 and i!=0:
            print("no of tuples classified: ")
            print(i)
          y_pred.append(self.classify(X.iloc[i]))  
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []
        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = posterior + prior
        # return class with the highest posterior
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    #def accuracy(self, y_true, y_pred):
        #accuracy = np.sum(str(y_true) == str(y_pred)) / len(y_true)
        #return accuracy

In [ ]:
import pandas as pd
path= "/content/drive/MyDrive/Dataasgn2 - Dataasgn2.csv"
data= pd.read_csv(path,header= None, names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "salary"])
# print(data)
data.drop(['fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week', 'age'], inplace=True, axis=1)
data.replace("?", np.nan)
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
data.replace('>50K', "-1", inplace=True)
data.replace('<=50K', "1", inplace=True)
data = data.dropna()
X_train= data.iloc[0:26049,:9]
y_train= data['salary'].iloc[:26049]
y_test= data['salary'].iloc[26050:]
X_test= data.iloc[26050:,:9]

In [ ]:
data

,workclass,education,education-num,marital-status,occupation,relationship,race,sex,native-country,salary
0,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,1
1,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1
2,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,1
3,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,1
4,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,1
...,...,...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,1
32557,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,-1
32558,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,United-States,1
32559,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,United-States,1


In [ ]:
print(y_train)
print(y_test)

0         1
1         1
2         1
3         1
4         1
         ..
26044     1
26045     1
26046     1
26047    -1
26048     1
Name: salary, Length: 26049, dtype: object
26050    -1
26051     1
26052     1
26053     1
26054     1
         ..
32556     1
32557    -1
32558     1
32559     1
32560    -1
Name: salary, Length: 6511, dtype: object


In [ ]:
    nb = NaiveBayes()
    nb.fit(X_train, y_train)

reached here


In [ ]:
print(nb._class_prob)

{'workclass': salary  workclass       
-1      ?                   0.005874
        Federal-gov         0.011133
        Local-gov           0.018734
        Private             0.150908
        Self-emp-inc        0.019079
        Self-emp-not-inc    0.022726
        State-gov           0.011133
1       ?                   0.050175
        Federal-gov         0.018388
        Local-gov           0.046566
        Never-worked        0.000192
        Private             0.544627
        Self-emp-inc        0.015356
        Self-emp-not-inc    0.056125
        State-gov           0.028600
        Without-pay         0.000384
dtype: float64, 'education': salary  education   
-1      10th            0.002265
        11th            0.002035
        12th            0.000998
        1st-4th         0.000192
        5th-6th         0.000422
        7th-8th         0.001267
        9th             0.000768
        Assoc-acdm      0.008062
        Assoc-voc       0.010941
        Bachelors     

In [ ]:
predictions = nb.predict(X_test)
# print("Naive Bayes classification accuracy", accuracy(y_test, predictions))

no of tuples classified: 
1000
no of tuples classified: 
2000
no of tuples classified: 
3000
no of tuples classified: 
4000
no of tuples classified: 
5000
no of tuples classified: 
6000


In [ ]:
print(predictions)

[-1  1  1 ...  1  1 -1]


In [ ]:
print(len(predictions))
print(len(y_test))

6511
6511


In [ ]:
import numpy as np
predictions = np.array(predictions)
y_test = np.array(y_test)
mis = 0
for i in range(len(predictions)):
  if str(predictions[i])==str(y_test[i]):
    mis+=1

print("accuracy is" , mis/len(predictions))

accuracy is 0.7061895254185225


In [102]:
import numpy as np
import statistics
predictions = np.array(predictions)
y_test = np.array(y_test)
tp = 0
for i in range(len(predictions)):
  if str(predictions[i]) == '1' and str(y_test[i]) == '1':
    tp += 1

fn=0
for i in range(len(predictions)):
  if str(predictions[i]) == '1' and str(y_test[i]) == '-1':
    fn += 1

fp=0
# m1 = 0
# p1 = 0
# for i in range(len(predictions)):
#   if str(predictions[i]) == '-1':
#     m1 += 1
#   elif str(predictions[i]) == '1':
#     p1 += 1
# print(m1)
# print(p1)


recall_den = tp + fn
precision_den = tp + fp
recall = tp/recall_den
precision = tp/precision_den
num_f1 = 2*recall*precision
den_f1 = recall + precision 


print("recall is" , recall)
print("precision is" , precision)
print("f1 score is" , num_f1/den_f1)


       
   


recall is 0.9640866873065016
precision is 1.0
f1 score is 0.9817150063051703
